In [56]:
#import libraries
import os
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import hvplot.pandas
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer


import os
import numpy as np
import pandas as pd
import random

In [57]:
# Set News API Key
newsapi = NewsApiClient(api_key='dc895cb5774c42ceaadd10ca782350dc')

In [58]:
# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp("2021-01-11", tz="America/New_York").isoformat()
past_date = pd.Timestamp("2020-12-10", tz="America/New_York").isoformat()


In [59]:
print(current_date)


2021-01-11T00:00:00-05:00


In [60]:
type(current_date)

str

In [61]:
#define a function to get articles using a keyword from newsapi 
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=date,
            to=date,
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [62]:
# Get election articles 
election_headlines, dates = get_headlines("election")

Fetching news about 'election'
******************************
retrieving news from: 2021-01-11 00:00:00
retrieving news from: 2021-01-10 00:00:00
retrieving news from: 2021-01-09 00:00:00
retrieving news from: 2021-01-08 00:00:00
retrieving news from: 2021-01-07 00:00:00
retrieving news from: 2021-01-06 00:00:00
retrieving news from: 2021-01-05 00:00:00
retrieving news from: 2021-01-04 00:00:00
retrieving news from: 2021-01-03 00:00:00
retrieving news from: 2021-01-02 00:00:00
retrieving news from: 2021-01-01 00:00:00
retrieving news from: 2020-12-31 00:00:00
retrieving news from: 2020-12-30 00:00:00
retrieving news from: 2020-12-29 00:00:00
retrieving news from: 2020-12-28 00:00:00
retrieving news from: 2020-12-27 00:00:00
retrieving news from: 2020-12-26 00:00:00
retrieving news from: 2020-12-25 00:00:00
retrieving news from: 2020-12-24 00:00:00
retrieving news from: 2020-12-23 00:00:00
retrieving news from: 2020-12-22 00:00:00
retrieving news from: 2020-12-21 00:00:00
retrieving new

NewsAPIException: {'status': 'error', 'code': 'parameterInvalid', 'message': 'You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2020-12-12, but you have requested 2020-12-12. You may need to upgrade to a paid plan.'}

In [ ]:
sid = SentimentIntensityAnalyzer()

# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [ ]:
# Get averages of each topics sentiment
election_avg = headline_sentiment_summarizer_avg(election_headlines)


In [ ]:
# Combine Sentiment Averages into DataFrame
election_sentiments = pd.DataFrame(
    {
        "election_avg": election_avg,
    }
)

election_sentiments.index = pd.to_datetime(dates)

display(election_sentiments)

In [63]:
#pull in data from previously pulled articles about vote and protest 
data = Path('protest_vote_sentiment.csv')
protest_vote_df = pd.read_csv(data, index_col=0)
protest_vote_df.head()

,vote_avg,protest_avg
2021-01-11,-0.042335,-0.039450
2021-01-10,-0.061795,-0.258725
2021-01-09,-0.178465,-0.143980
2021-01-08,-0.147315,-0.362070
2021-01-07,-0.338450,-0.361915


In [64]:
protest_vote_df.shape, election_sentiments.shape

NameError: name 'election_sentiments' is not defined

In [ ]:
#merge dataframes with the election sentiment results 
news_sentiment = protest_vote_df.join(election_sentiments, how='inner')

In [ ]:
news_sentiment

In [ ]:
#export to a new csv
news_sentiment.to_csv("news_sentiment.csv")

In [ ]:
#plot the sentiment of the three key words
news_sentiment.hvplot()